In [1]:
from keras.models import Model
import keras.models
import keras.layers as layers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from keras import backend as K

import argparse
import copy
import feature_interpretation as cnna
import cnn_builder as cbuild
import cnn_runner as crun
import config
import csv
import niftiutils.helper_fxns as hf
import niftiutils.transforms as tr
import importlib
import itertools
from math import sqrt, log, pi, exp
import matplotlib.pyplot as plt
from numba import njit
from numpy import matmul, diag
import numpy as np
import operator
import os
from os.path import *
import pandas as pd
import random
import scipy
import time
import dr_methods as drm
import voi_methods as vm
%matplotlib inline
np.set_printoptions(3, suppress=True)

C:\Users\Clinton\AppData\Local\conda\conda\envs\old-keras\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
importlib.reload(config)
importlib.reload(hf)
importlib.reload(cbuild)
importlib.reload(crun)
C = config.Config()
T = config.Hyperparams()
T.get_best_hyperparams()
T.epochs = 30
T.steps_per_epoch = 300

In [36]:
T.padding = ['same', 'same']
T.pool_sizes = [(2,2,2),(2,2,2)]
T.f = [64,64,64,64,64,64,64]
T.skip_con = True
T.epochs = 20

In [ ]:
drm.dcm2npy_batch(acc_nums=["E100113043"])

In [ ]:
vm.plot_check(2, "E102088195");

In [ ]:
vm.xref_dirs_with_excel()

In [ ]:
crun.run_fixed_hyperparams([C], hyperparams=T)#C_list)


C = config.Config()
ActivationLayer = layers.Activation
activation_args = 'relu'

nb_classes = len(C.classes_to_include)

context_img = layers.Input(shape=(C.context_dims[0], C.context_dims[1], C.context_dims[2], 3))
cx = context_img

#cx = InstanceNormalization(axis=4)(cx)
cx = layers.Reshape((*C.context_dims, 3, 1))(context_img)
cx = layers.Permute((4,1,2,3,5))(cx)
cx = layers.TimeDistributed(layers.Conv3D(filters=64, kernel_size=(4,4,1), padding='valid', activation='relu'))(cx)
cx = layers.TimeDistributed(layers.MaxPooling3D((2,2,2)))(cx)
cx = layers.TimeDistributed(layers.BatchNormalization(axis=4))(cx)
cx = layers.TimeDistributed(layers.Conv3D(filters=64, kernel_size=(3,3,1), padding='valid', activation='relu'))(cx)
cx = layers.TimeDistributed(layers.BatchNormalization(axis=4))(cx)
cx = layers.TimeDistributed(layers.Conv3D(filters=64, kernel_size=(3,3,1), padding='valid', activation='relu'))(cx)
cx = layers.TimeDistributed(layers.BatchNormalization(axis=4))(cx)
cx = layers.TimeDistributed(layers.Conv3D(filters=64, kernel_size=(3,3,1), padding='same', activation='relu'))(cx)
cx = layers.TimeDistributed(layers.BatchNormalization(axis=4))(cx)

model = Model(context_img, cx)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
importlib.reload(cbuild)
model = cbuild.build_cnn_hyperparams(T)
model.summary(line_length=120)

________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
input_2 (InputLayer)                   (None, 24, 24, 12, 3)      0                                                     
________________________________________________________________________________________________________________________
lambda_8 (Lambda)                      (None, 24, 24, 12, 1)      0             input_2[0][0]                           
________________________________________________________________________________________________________________________
lambda_9 (Lambda)                      (None, 24, 24, 12, 1)      0             input_2[0][0]                           
________________________________________________________________________________________________________________________
lambda_10 (Lambda)              

In [9]:
importlib.reload(cbuild)
#Z_reader = ['E103312835_1','12823036_0','12569915_0','E102093118_0','E102782525_0','12799652_0','E100894274_0','12874178_3','E100314676_0','12842070_0','13092836_2','12239783_0','12783467_0','13092966_0','E100962970_0','E100183257_1','E102634440_0','E106182827_0','12582632_0','E100121654_0','E100407633_0','E105310461_0','12788616_0','E101225606_0','12678910_1','E101083458_1','12324408_0','13031955_0','E101415263_0','E103192914_0','12888679_2','E106096969_0','E100192709_1','13112385_1','E100718398_0','12207268_0','E105244287_0','E102095465_0','E102613189_0','12961059_0','11907521_0','E105311123_0','12552705_0','E100610622_0','12975280_0','E105918926_0','E103020139_1','E101069048_1','E105427046_0','13028374_0','E100262351_0','12302576_0','12451831_0','E102929168_0','E100383453_0','E105344747_0','12569826_0','E100168661_0','12530153_0','E104697262_0']
X_test, Y_test, train_generator, num_samples, train_orig, Z = cbuild.get_cnn_data(n=4)#, Z_test_fixed=Z_reader)
Z_test, Z_train_orig = Z
X_train_orig, Y_train_orig = train_orig
hist = model.fit_generator(train_generator, steps_per_epoch=T.steps_per_epoch, epochs=T.epochs, validation_data=[X_test, Y_test])#, callbacks=[T.early_stopping])

Epoch 1/30
300/300 [==============================] - 13s 43ms/step - loss: 0.2921 - acc_logit: 0.8796 - val_loss: 0.5480 - val_acc_logit: 0.7000
Epoch 2/30
300/300 [==============================] - 11s 38ms/step - loss: 0.2176 - acc_logit: 0.9129 - val_loss: 0.4994 - val_acc_logit: 0.9000
Epoch 3/30
300/300 [==============================] - 11s 38ms/step - loss: 0.2076 - acc_logit: 0.9146 - val_loss: 0.7892 - val_acc_logit: 0.8000
Epoch 4/30
300/300 [==============================] - 11s 38ms/step - loss: 0.1856 - acc_logit: 0.9283 - val_loss: 0.8538 - val_acc_logit: 0.9000
Epoch 5/30
300/300 [==============================] - 12s 39ms/step - loss: 0.1513 - acc_logit: 0.9383 - val_loss: 0.2870 - val_acc_logit: 0.8000
Epoch 6/30
300/300 [==============================] - 12s 39ms/step - loss: 0.1600 - acc_logit: 0.9296 - val_loss: 0.4918 - val_acc_logit: 0.8000
Epoch 7/30
300/300 [==============================] - 12s 38ms/step - loss: 0.1491 - acc_logit: 0.9392 - val_loss: 1.4876 - 

In [10]:
model.save(join(C.model_dir, "prob_gauss.hdf5"))

In [ ]:
model.predict(X_train_orig[20:40])

In [ ]:
Y_train_orig[:20]

In [ ]:
model = cbuild.build_cnn_demogr()
model.summary(line_length=120)
X_test, Y_test, train_generator, num_samples, train_orig = cbuild.get_cnn_demogr(n=4)
hist = model.fit_generator(train_generator, steps_per_epoch=T.steps_per_epoch, epochs=T.epochs, validation_data=[X_test, Y_test])#, callbacks=[T.early_stopping])

In [6]:
#from keras.utils import multi_gpu_model
#model = multi_gpu_model(model, gpus=2)

In [ ]:
import voi_methods as vm
vm.reset_accnum('E105464882')

In [ ]:
hist = model.fit_generator(train_generator, steps_per_epoch=T.steps_per_epoch, epochs=T.epochs, validation_data=[X_test, Y_test])#, callbacks=[T.early_stopping])

In [82]:
model.save('E:\\models\\model_demogr.hdf5')

In [ ]:
import dr_methods as drm
voi_df_art, voi_df_ven, voi_df_eq = drm.get_voi_dfs()
small_voi_df = pd.read_csv(C.small_voi_path)

In [ ]:
#plot_with_bbox(fn_list[2], cls_mapping[wrong_guesses[2]])
Y_pred = model.predict(X_test)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_test])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

#save_output(Z_test, y_pred, y_true)

print(accuracy_score(y_true, y_pred))
#y_true_simp, y_pred_simp, _ = cnna.merge_classes(y_true, y_pred)
#print(accuracy_score(y_true_simp, y_pred_simp))

In [90]:
importlib.reload(cbuild)

<module 'cnn_builder' from 'C:\\Users\\Clinton\\Documents\\voi-classifier\\python\\cnn_builder.py'>

In [3]:
model = keras.models.load_model(os.path.join(C.model_dir, "models_305.hdf5")) #models_305

In [ ]:
#model_frozen = cbuild.build_frozen_model(model, last_layer=-2)

In [ ]:
X_test, Y_test, train_generator, num_samples, train_orig, Z = cbuild.get_cnn_data(n=4, n_art=0, Z_test=Z_reader)
Z_test, Z_train_orig = Z
Y_pred = model.predict(X_test)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_test])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])
y_true_simp, y_pred_simp, _ = cbuild.merge_classes(y_true, y_pred)

In [ ]:
Y_pred = [C.classes_to_include[y] for y in y_pred]
Y_true = [C.classes_to_include[y] for y in y_true]

In [ ]:
a = pd.DataFrame([Z_test,Y_pred,Y_true])

In [ ]:
a.to_csv('E:\\temp.csv')

In [ ]:
acc6cls = []
acc3cls = []

for i in range(19):
    model_num = 306+i
    X_test, Y_test, train_generator, num_samples, train_orig, Z = cbuild.get_cnn_data(n=4, n_art=0, Z_test=df[df['model_num'] == model_num]["z_test"])
    X_train_orig, Y_train_orig = train_orig
    model = keras.models.load_model(os.path.join(C.model_dir, "models_%d.hdf5" % model_num)) #models_305
    
    Y_pred = model.predict(X_train_orig)
    y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_train_orig])
    y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

    acc6cls.append(accuracy_score(y_true, y_pred))

    y_true_simp, y_pred_simp, _ = cbuild.merge_classes(y_true, y_pred)

    acc3cls.append(accuracy_score(y_true_simp, y_pred_simp))

In [ ]:
df = pd.read_excel("Z:\\Inter-reader study\\Answer key.xlsx")

In [ ]:
y_true = df["Class"].values
y_pred = df["Model"].values

In [ ]:
cm = confusion_matrix(y_true, y_pred, labels=C.classes_to_include)

In [ ]:
for i in range(len(cm)):
    print('\t'.join(cm[:,i].astype(str)))

In [ ]:
fn_list = fn_list + list(Z_test[~np.equal(y_pred, y_true)])

In [ ]:
fn_list = fn_list + list(Z_train_orig[~np.equal(y_pred, y_true)])
fn_list = fn_list + list(Z_train_orig[~np.equal(y_pred, y_true)])

In [ ]:
fn_x_list = [x+"("+str(voi_df_art[voi_df_art["id"] == x[:-4]]["x1"].values[0])+")" for x in fn_list]

In [ ]:
unique, counts = np.unique(fn_x_list, return_counts=True)

In [ ]:
dict(zip(unique, counts))

In [195]:
Y_pred = model.predict(X_train_orig)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_train_orig])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

#cbuild.save_output(Z_train_orig, y_pred, y_true)#, save_dir=C.output_img_dir+"\\training")

In [134]:
importlib.reload(cbuild)

<module 'cnn_builder' from 'C:\\Users\\Clinton\\Documents\\voi-classifier\\python\\cnn_builder.py'>